In [1]:
from __future__ import division, print_function
%matplotlib inline
#from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *

Using TensorFlow backend.


In [2]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [3]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

491520/600901 [=======================>......] - ETA: 1scorpus length: 600901


In [4]:
!tail -n 25 {path}

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 60


In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
idx = [char_indices[c] for c in text]

In [10]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [11]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [12]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600862


In [13]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [14]:
sentences.shape, next_chars.shape

((600860, 40), (600860, 40))

In [15]:
n_fac = 24

In [16]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(units=512, input_shape=(n_fac,),return_sequences=True, dropout=0.2, recurrent_dropout=0.2,
             implementation=2),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2,
             implementation=2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [18]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]  # [-40] picks up the last 40 chars
        preds = model.predict(x, verbose=0)[0][-1]  # [-1] picks up the last char
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [19]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

Epoch 1/1
173696/600860 [=======>......................] - ETA: 702s - loss: 1.8228

KeyboardInterrupt: 

In [ ]:
print_example()

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

In [ ]:
print_example()

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

In [ ]:
print_example()

In [ ]:
model.optimizer.lr=0.0001

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

In [ ]:
print_example()

In [ ]:
model.save_weights('data/char_rnn.h5')

In [ ]:
model.optimizer.lr=0.00001

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

In [ ]:
print_example()

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

In [ ]:
print_example()

In [ ]:
print_example()

In [ ]:
model.save_weights('data/char_rnn.h5')